# Carga de datos

## Librerias

In [1]:
from os import path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Constantes

In [2]:
SAVE_DATAFRAME = False
SAVE_FIG = False

PATH_MAIN = path.join("..")
PATH_DATA = path.join(PATH_MAIN, "datos")

PATH_FAKE = path.join(PATH_DATA, "Fake.csv")
PATH_TRUE = path.join(PATH_DATA, "True.csv")

## Carga del dataframe

En esta sección cargamos los DataFrames y además le ponemos una etiqueta de que si es una noticia verdadera o falsa, para posteriormente unir los DataFrames.

### DataFrame de True

In [3]:
# Nombre de la columna que almacenará la 
# información de si es una noticia verdadera o falsa
name_col = "true"

In [4]:
# Cargamos DataFrame
df_true = pd.read_csv(PATH_TRUE)

# Agregamos nueva columna
df_true[name_col] = 1

df_true.head()

,title,text,subject,date,true
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


### DataFrame de Fake

In [5]:
# Cargamos DataFrame
df_fake = pd.read_csv(PATH_FAKE)

# Agregamos nueva columna
df_fake[name_col] = 0

del name_col

df_fake.head()

,title,text,subject,date,true
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


### Merge entre DataFrames
Unimos los DataFrames, ahora que se tiene una indexización.

In [6]:
df = df_true.append(df_fake, ignore_index=True)
df.head()

,title,text,subject,date,true
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


### Columnas del DataFrame

In [7]:
# Solo basta un dataframe pues comparten el nombre de las columnas
columnas = list(df.columns)
columnas

['title', 'text', 'subject', 'date', 'true']

## Información de los datasets

### Información de los datasets

Visualizamos la información de las filas de los datasets.

In [8]:
df_true.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21417 entries, 0 to 21416
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    21417 non-null  object
 1   text     21417 non-null  object
 2   subject  21417 non-null  object
 3   date     21417 non-null  object
 4   true     21417 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 836.7+ KB


In [9]:
df_fake.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23481 entries, 0 to 23480
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    23481 non-null  object
 1   text     23481 non-null  object
 2   subject  23481 non-null  object
 3   date     23481 non-null  object
 4   true     23481 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 917.4+ KB


In [10]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44898 entries, 0 to 44897
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    44898 non-null  object
 1   text     44898 non-null  object
 2   subject  44898 non-null  object
 3   date     44898 non-null  object
 4   true     44898 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 1.7+ MB


### Eliminar los datos que no sean fechas
En algunas filas de los DataFrames hay información etiquetada como fecha, cuando no corresponde a una fecha. Reemplazamos estos valores por un tipo de dato NaT y los dropeamos.

In [11]:
def is_date(string, fuzzy=False):
    """
    Return whether the string can be interpreted as a date.

    :Param string: str, string to check for date
    :Param fuzzy: bool, ignore unknown tokens in string if True
    """
    from dateutil.parser import parse
    
    try: 
        parse(string, fuzzy=fuzzy)
        return True

    except ValueError:
        return False

In [12]:
nat = np.datetime64('NaT')

def nat_conversor(x): return pd.Timestamp(x) if is_date(x) else nat

df_fake["date"] = df_fake["date"].apply(nat_conversor)
df_true["date"] = df_true["date"].apply(nat_conversor)
df["date"] = df["date"].apply(nat_conversor)

L_df = [df_fake, df_true, df]

In [15]:
for df_ in L_df:
    df_.dropna(inplace=True)

del nat_conversor, nat

### Variables únicas

En principio, la columna `'title'` y `'text'` no son ni númericas, ni categóricas. `'date'` es un tipo de variable temporal, mientras que `'subject'` es una variable categórica. 

Veamos cuantas variables categóricas diferentes hay para cada `DataFrame`.

In [32]:
print("El dataset de noticias verdaderas posee las siguientes etiquetas:")
for label in pd.unique(df_true.subject):
    print(f"* {label}")
    
print("\nEl dataset de noticias falsas posee las siguientes etiquetas:")
for label in pd.unique(df_fake.subject):
    print(f"* {label}")

El dataset de noticias verdaderas posee las siguientes etiquetas:
* politicsNews
* worldnews

El dataset de noticias falsas posee las siguientes etiquetas:
* News
* politics
* Government News
* left-news
* US_News
* Middle-east


In [33]:
def imprimir_cantidad_por_categoria(df_, categoria):
    cantidad = len(df_[df_.subject == categoria])
    print(f"La categoría {categoria} posee {cantidad} datos.")
    return None

print("Cantidad de datos por categoría para las noticias verdaderas:")
for label in pd.unique(df_true.subject):
    imprimir_cantidad_por_categoria(df_true, label)

    
print("\nCantidad de datos por categoría para las noticias falsas:")
for label in pd.unique(df_fake.subject):
    imprimir_cantidad_por_categoria(df_fake, label)

Cantidad de datos por categoría para las noticias verdaderas:
La categoría politicsNews posee 11272 datos.
La categoría worldnews posee 10145 datos.

Cantidad de datos por categoría para las noticias falsas:
La categoría News posee 9050 datos.
La categoría politics posee 6836 datos.
La categoría Government News posee 1568 datos.
La categoría left-news posee 4456 datos.
La categoría US_News posee 783 datos.
La categoría Middle-east posee 778 datos.


In [26]:
len(df_true[df_true.subject == 'worldnews'])

10145

¿A qué se refiere `'worldnews'`? Obtengamos una noticia de esta categoría para observarlo:

In [18]:
def revisar_noticias(df, category, gap=5):
    """Entrega varias noticias de una categoría en específico.
    'df' es el DataFrame, 'category' es la categoría que se quiere revisar
    y 'gap' es el número de noticias que se quiere mostrar.
    """
    df_ = df[df.subject == category]
    df_.reset_index(inplace=True)
    for i in range(gap):
        print("Título:", f'"{df_.title[i]}"')
        print("Texto:", f"{df_.text[i]}")
        print("\n", "-" * 80, "\n")

In [19]:
revisar_noticias(df_true, "worldnews")

Título: "Reuters journalists in Myanmar appear in court, remanded for another 14 days"
Texto: YANGON (Reuters) - Two Reuters journalists who have been detained in Myanmar for the past two weeks were remanded in custody for a further two weeks on Wednesday as a probe continues into allegations they breached the nation s Official Secrets Act. Judge Ohn Myint granted the 14-day extension in the case of the journalists, Wa Lone, 31, and Kyaw Soe Oo, 27, at the request of the police, who then took them to Yangon s Insein prison. They were previously being held in a police compound. When they appeared at the Mingaladon court for the proceedings, Wa Lone and Kyaw Soe Oo were allowed to meet their families and their lawyer for the first time since their arrest. The two journalists had worked on Reuters coverage of a crisis in the western state of Rakhine, where - according to United Nations  estimates - about 655,000 Rohingya Muslims have fled from a fierce military crackdown on militants. The

Podemos decir entonces que una noticia de `'worldnews'` es una noticia estadounidense, que se refiere c/r al resto del mundo mundo.

¿Existirá un equivalente en `fakenews`? Empecemos por revisar la categoría `News`.

In [20]:
revisar_noticias(df_fake, "News")

Título: " Donald Trump Sends Out Embarrassing New Year’s Eve Message; This is Disturbing"
Texto: Donald Trump just couldn t wish all Americans a Happy New Year and leave it at that. Instead, he had to give a shout out to his enemies, haters and  the very dishonest fake news media.  The former reality show star had just one job to do and he couldn t do it. As our Country rapidly grows stronger and smarter, I want to wish all of my friends, supporters, enemies, haters, and even the very dishonest Fake News Media, a Happy and Healthy New Year,  President Angry Pants tweeted.  2018 will be a great year for America! As our Country rapidly grows stronger and smarter, I want to wish all of my friends, supporters, enemies, haters, and even the very dishonest Fake News Media, a Happy and Healthy New Year. 2018 will be a great year for America!  Donald J. Trump (@realDonaldTrump) December 31, 2017Trump s tweet went down about as welll as you d expect.What kind of president sends a New Year s gre

Parecen ser simplemente noticias estándar, nada relacionado con noticias del mundo.

In [21]:
revisar_noticias(df_fake, "politics")

Título: "Democrat Senator Warns Mueller Not To Release Findings On Russia Before 2018 Midterms"
Texto: According to The Hill, Democrat Senator Bob Casey is warning special counsel Robert Mueller against issuing his Russia report near the 2018 electionSen. Bob Casey Jr. (D-Pa.) on Sunday warned special counsel Robert Mueller against issuing a report on his findings in the ongoing investigation of Russian interference in the 2016 election near the upcoming 2018 midterm elections.Speaking to radio host John Catsimatidis on New York radio station AM 970, Casey said he couldn t make any assumptions about where the Mueller investigation is going in light of indictments issued on Friday. But he added that he would recommend Mueller not release a report on his findings near the midterms when it would distract from elections or cause people to question the election s integrity. I don t think we ll know anywhere near the full story until [Mueller] issues his report,  Casey said.  But once you ge



Revisemos con la categoría `Middle-east`

In [ ]:
revisar_noticias(df_fake, "Middle-east")

No parece tener relación con noticias del mundo. Veamos ahora la categoría `left-news`

In [ ]:
revisar_noticias(df_fake, 'left-news')

##### Conclusión de Panchito

Con la última categoría tentativa a ser una noticia acerca del mundo, podemos concluir a priori que no hay noticias falsas acerca del resto del mundo. Con lo que una alternativa tentativa para que el algoritmo no sufra de overfitting, es el de eliminar esta categoría.

El problema que tendríamos con esto, es quedarnos únicamente con noticias acerca de la política, así que una opción a esto sería eliminar varias categorías de las noticias falsas, o bien, mantenerlas y decidir cuál específicamente son de política.

Otra opción es predecir sin ocupar esta columna, para no tener ese "spolier" de ser una noticia falsa.

# EDA

### Análisis gráfico del tiempo

In [ ]:
def rotular(ax, title, xlabel, ylabel):
    """Rotula un gráfico
    """
    ax.set_title(title)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)

In [ ]:
def my_plot(ts_t, ts_f, 
            title=None, xlabel=None, ylabel=None,
            rotation=40):
    """Plotea las cosas bajo el contexto que necesitamos.
    """
    fig, ax = plt.subplots()  # Create a figure containing a single axes.
    ax.plot(ts_f)  # Plot some data on the axes.
    ax.plot(ts_t)
    ax.legend(['Falsa', 'Verdadera'], loc="best")

    plt.xticks(rotation=rotation)

    rotular(ax, 
            title=title,
            xlabel=xlabel,
            ylabel=ylabel,)

    plt.show()

In [ ]:
L_dow = ["Monday", "Tuesday", "Wednesday",
         "Thursday", "Friday", "Saturday", "Sunday", ]


def count_dow(ser):
    """Cuenta los días de la semana y los rotúla.
    """
    s = ser.dt.day_name().value_counts()
    s = pd.Series({L_dow[i]: s[L_dow[i]] for i in range(7)})
    return s


ts_t = count_dow(df_true["date"])
ts_f = count_dow(df_fake["date"])

my_plot(ts_t, ts_f,
        title="Comparación número de noticias por día de semana",
        xlabel="Día de la semana",
        ylabel="Número de noticias",)

Observamos que las noticias verdaderas sacan noticas de forma constante, mientras que las noticias falsas tiene una baja de más del 50% en los fines de semana.

In [ ]:
m = {
    1: "January", 2: "February", 3: "March", 4: "April",
    5: "May", 6: "June", 7: "July", 8: "August",
    9: "September", 10: "Octuber", 11: "November", 12: "December"
}

def number2month(n):
    """Hace un mapeo del número de un mes, al nombre del mes.
    Retorna None si no se entrega un número entero entre el 1 y el 12.
    """
    return m.get(n, None)

def count_month(ser):
    """Cuenta los meses y los rotúla.
    """
    s = ser.dt.month.map(number2month).value_counts()
    s = pd.Series({m[i]:s[m[i]] for i in range(1, 13)})
    return s

ts_f = count_month(df_fake["date"])
ts_t = count_month(df_true["date"])

my_plot(ts_t, ts_f, rotation=60,
        title="Comparación número de noticias por mes",
        xlabel="Mes",
        ylabel="Número de noticias",)

Vemos que las noticias verdaderas se mantienen constantes en los meses, mientras que las noticias falsas se mantienen por debajo de las noticias verdaderas. Sin embargo, esta situación cambia en los últimos meses a partir de agosto. Se puede deber que en esos meses empezaron las presidenciales.

In [ ]:

# ts_f = df_fake["date"].apply(lambda x: x.month).value_counts(
# ).reset_index().sort_values(by=['index'])
# ts_t = df_true["date"].apply(lambda x: x.month).value_counts(
# ).reset_index().sort_values(by=['index'])

# fig, ax = plt.subplots()  # Create a figure containing a single axes.
# ax.plot(ts_f['index'], ts_f['date'])  # Plot some data on the axes.
# ax.plot(ts_t['index'], ts_t['date'])
# fig.legend(['Verdadera', 'Falsa'])

# plt.show()

In [ ]:
# m = {
#     1: "January", 2: "February", 3: "March", 4: "April",
#     5: "May", 6: "June", 7: "July", 8: "August",
#     9: "September", 10: "Octuber", 11: "November", 12: "December"
# }

# def number2month(n):
#     """Hace un mapeo del número de un mes, al nombre del mes.
#     Retorna None si no se entrega un número entero entre el 1 y el 12.
#     """
#     return m.get(n, None)

# def count_year(ser):
#     """Cuenta los meses y los rotúla.
#     """
#     s = ser.dt.year.map(str).value_counts()#.reset_index().sort_values(by=['index'])
# #     s = pd.Series({m[i]:s[m[i]] for i in range(1, 13)})
#     return s

In [ ]:
# ts_f = count_year(df_fake["date"])
# ts_t = count_year(df_true["date"])

# my_plot(ts_t, ts_f, rotation=60,
#         title="Comparación número de noticias por mes",
#         xlabel="Mes",
#         ylabel="Número de noticias",)

In [ ]:
ts_f = df_fake["date"].dropna().apply(lambda x: pd.Timestamp(
    x.strftime("%Y-%m"))).value_counts().reset_index().sort_values(by=['index'])
ts_t = df_true["date"].dropna().apply(lambda x: pd.Timestamp(
    x.strftime("%Y-%m"))).value_counts().reset_index().sort_values(by=['index'])

# Create a figure containing a single axes.
fig, ax = plt.subplots(figsize=(15, 7))
ax.plot(ts_f['index'], ts_f['date'], label= 'Falsa')  # Plot some data on the axes.
ax.plot(ts_t['index'], ts_t['date'], label= 'Verdadera')
fig.legend()

plt
plt.show()

## Nubes de palabras

Se procederá a elaborar las nubes de palabras a partir del cuerpo y el título de las noticias. 

In [ ]:
# Importamos las funciones

from nltk.probability import FreqDist
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
stop_words=set(stopwords.words("english"))
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

tokenizer = RegexpTokenizer(r"\w+")
lem = WordNetLemmatizer()
stem = PorterStemmer()
stop_words.add('thi')

In [ ]:
# Agregamos los signos de puntuación a los stop_words

In [ ]:
# Variables de los nombres de las columnas

TITLE = 'title'
TEXT= 'text'
SUBJECT = 'subject'
DATE = 'date'
TRUE= 'true'

In [ ]:
def show_wordcloud(data, title = None):
    """
    Retorna la visualización de una nube de palabras.

    :Param data: str, string del texto que se busca visualizar. 
    :Param title: str, título de la figura generada.
    """
    
    wordcloud = WordCloud(
        background_color='white',
        stopwords=stop_words,
        max_words=200,
        max_font_size=40, 
        scale=3,
        random_state=1 # chosen at random by flipping a coin; it was heads
    ).generate(str(data))

    fig = plt.figure(1, figsize=(12, 12))
    plt.axis('off')
    if title: 
        fig.suptitle(title, fontsize=20)
        fig.subplots_adjust(top=2.3)

    plt.imshow(wordcloud)
    plt.show()

### Titulares

In [ ]:
# Titulares noticias falsas
FAKE_TITLE = []
for i in df[df[TRUE] == 0].index:
    FAKE_TITLE.append(df[TITLE][i])

# Titulares noticias verdaderas
TRUE_TITLE = []
for i in df[df[TRUE]== 1].index:
    TRUE_TITLE.append(df[TITLE][i])
    
# Cada elemento de las listas anteriores en la oración original del titular. Ahora debemos tokenizar las oraciones, eliminar
# las stopwords y steamizarlas. 

# Titulares noticias falsas transformados
FAKE_=[]
for i in FAKE_TITLE:
    for j in tokenizer.tokenize(i):
        if j.lower() not in stop_words:
            FAKE_.append(stem.stem(j))
FAKE_TITLE= FAKE_

# Titulares noticias verdaderas transformados
TRUE_=[]
for i in TRUE_TITLE:
    for j in tokenizer.tokenize(i):
        if j.lower() not in stop_words:
            TRUE_.append(stem.stem(j))
TRUE_TITLE= TRUE_

# Ahora para que el formato de entrada a la función que genera las nubes de palabras sea el correcto, debemos unir la lista 
# en un solo string. 

FAKE_TITLE= ' '.join(FAKE_TITLE)
TRUE_TITLE= ' '.join(TRUE_TITLE) 

In [ ]:
show_wordcloud(TRUE_TITLE, title= 'Titulares de noticias verdaderas')

In [ ]:
show_wordcloud(FAKE_TITLE, title= 'Titulares de noticias falsas')

## Cuerpo de la noticia

In [ ]:
# Texto noticias falsas
FAKE_TEXT = []
for i in df[df[TRUE] == 0].index:
    FAKE_TEXT.append(df[TEXT][i])

# Texto noticias verdaderas
TRUE_TEXT = []
for i in df[df[TRUE]== 1].index:
    TRUE_TEXT.append(df[TEXT][i])
    
# Cada elemento de las listas anteriores es el texto original del cuerpo de la noticia. Ahora debemos tokenizar las 
# oraciones, eliminar las stopwords y steamizarlas. 

# Titulares noticias falsas transformados
FAKE_T=[]
for i in FAKE_TEXT:
    for j in tokenizer.tokenize(i):
        if j.lower() not in stop_words:
            FAKE_T.append(stem.stem(j))
FAKE_TEXT= FAKE_T

# Titulares noticias verdaderas transformados
TRUE_T=[]
for i in TRUE_TEXT:
    for j in tokenizer.tokenize(i):
        if j.lower() not in stop_words:
            TRUE_T.append(stem.stem(j))
TRUE_TEXT= TRUE_T

# Ahora para que el formato de entrada a la función que genera las nubes de palabras sea el correcto, debemos unir la lista 
# en un solo string. 

FAKE_TEXT= ' '.join(FAKE_TEXT)
TRUE_TEXT= ' '.join(TRUE_TEXT) 

In [ ]:
show_wordcloud(TRUE_TEXT, title= 'Cuerpo de noticias verdaderas')

In [ ]:
show_wordcloud(FAKE_TEXT, title= 'Cuerpo de noticias falsas')

Vale: Lo intenté, pero mi compu quedó K.O. Por lo menos el código tiene toda la pinta de funcionar ajjshja

### Tutorial nltk 

Vale: Ya hice algunos imports y le cambié el nombre a algunas funciones que tenían nombre muy largo. Así que ahora lo voy a explicar: 

tokenizer = RegexpTokenizer(r"\w+") 

Lo que hace es separar un string de un párrafo u oración en una lista de strings con las palabras. La diferencia con la función 'word_tokenize', es que RegexpTokenize puede configurarse para aceptar solamente ciertos caracteres. El argumento r"\w+" es para que ignore la puntuación del texto y no la incluya en la lista. Sé que también se puede configurar para ignorar los números, habría que buscar en google el argumento correcto. 

stem = PorterStemmer()

Lo que hace es devolver una palabra o verbo conjugado a su palabra raíz.

lem = WordNetLemmatizer()

Lo que hace es devolver una palabra o verbo conjugado a su palabra raíz. Dependiendo del segundo argumento que reciba (el primer argumento en el texto como string) puede definir las raices como verbos, pronombres, etc. A diferencia de steamming, relaciona las palabras con su significado, no solamente a partir de la gramática. Esta página (https://www.machinelearningplus.com/nlp/lemmatization-examples-python/) explica que para hacerlo bien hay que usar POS tag, que es otra función que detecta si la palabra es adjetivo, adverbio, sustantivo o verbo, y así WordNetLemmatizer() se puede llamar de forma adecuada para cada palabra.

stop_words=set(stopwords.words("english"))

Es un set con los pronombres, preposiciones, y otro tipo de palabras que no aportan valor. Usando add se pueden incluir palabras nuevas. 

A continuación un ejemplo de uso. 

In [ ]:
txt= df[TITLE][325]

print ( 'El texto original es: ' + txt + '\n')

token_txt= tokenizer.tokenize(txt)

print('El texto toquenizado es: ' )
print(token_txt)
print('\n')

stop_txt=[]
for i in token_txt: 
    if i not in stop_words:
        stop_txt.append(i)

print('El texto sin stop words es: ' )
print(stop_txt)
print('\n')

stem_txt= []
for i in stop_txt:
    stem_txt.append(stem.stem(i))
    
print('El texto stemizado es: ' )
print(stem_txt)  


PD: Cuando estaba programando las nubes de palabras, me di cuenta de que en las noticias falsas, los títulos van siempre con la primera letra de cada palabra en mayúscula (y en las noticias verdaderas no pasa). Podríamos hacer una predicción sin cambiar eso, para ver si considera las mayúsculas como importantes y después otra usando todo en minúscula, porque igual es la media trampa. 